In [ ]:
pip install arabert

In [1]:
import pandas as pd 
import torch
from transformers import BertTokenizer, BertForMaskedLM, AdamW, BartForConditionalGeneration
from torch.utils.data import DataLoader, TensorDataset , Dataset
from sklearn.model_selection import train_test_split
import re
import numpy as np
import random
import nltk
nltk.download('punkt')
from arabert import ArabertPreprocessor
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel
from transformers import AutoTokenizer, AutoModelForMaskedLM , AutoConfig
import os

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def split_sentences_with_overlap(sentences, num_words_per_split=32):

    split_sentences = []
    

    for sentence in sentences:

        words = sentence.split()
        start_word_index = 0


        for i in range(len(words)):

            end_word_index = min(len(words), start_word_index + num_words_per_split)

            current_part = words[start_word_index:end_word_index]
            if len(current_part) >= num_words_per_split or i == len(words) - 1:

 
                split_sentences.append(' '.join(current_part))
                

                start_word_index = start_word_index + num_words_per_split

    return split_sentences

In [3]:
arabic_letters = "ابتثجحخدذرزسشصضطظعغفقكلمنهوي"

def misspelling_word(article, misspelling_ratio=0.45):
    
    words = article.split()

    for i in range(len(words)):
        if random.random() < misspelling_ratio:

            word = words[i]

            operation = random.randint(0, 2)
            
            if len(word) > 3:

                position = random.randint(0, len(word) - 1)
                
                if operation == 0:  
                    
                    random_letter = random.choice(arabic_letters.replace(word[position], ''))

                    word = word[:position] + random_letter + word[position + 1:]
                    
                elif operation == 1: 
                    
                    word = word[:position] + word[position + 1:]
                    
                elif operation == 2: 
                    random_letter = random.choice(arabic_letters)

                    word = word[:position] + random_letter + word[position:]


            words[i] = word

    modified_article = ' '.join(words)
    return modified_article

In [4]:
def process_folder(folder_path, max_articles):
    input_articles = []
    target_articles = []
    num_articles_processed = 0

    for filename in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, filename)):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                input_article = file.read()
                input_article = re.sub(r'[^\u0621-\u064A ]+', '', input_article)
                target_article = input_article  # Assuming target is the same as input for now
                input_article = misspelling_word(input_article)
                input_articles.append(input_article)
                target_articles.append(target_article)
                num_articles_processed += 1

                if num_articles_processed >= max_articles:
                    break

    return input_articles, target_articles

def process_folders(root_directory, max_articles_per_folder=600):
    input_articles = []
    target_articles = []

    for folder_name in os.listdir(root_directory):
        folder_path = os.path.join(root_directory, folder_name)
        if os.path.isdir(folder_path):
            input_articles_folder, target_articles_folder = process_folder(folder_path, max_articles=max_articles_per_folder)
            input_articles.extend(input_articles_folder)
            target_articles.extend(target_articles_folder)

    return input_articles, target_articles

root_directory = '/kaggle/input/sanad-dataset'
input_articles, target_articles = process_folders(root_directory)



split_input_articles = split_sentences_with_overlap(input_articles)
split_target_articles = split_sentences_with_overlap(target_articles)

In [24]:
split_input_articles[30000:30010]

['الجنتيهات وصب الفثير من اللاعبين والمدربين اميرات مراقبة في ممتلكاتهم كما صمم بعضكم غطرفة محصقنة في المنزل أو ما يعرف بغرفة الهلع وتردد أن وكلاء غللاعبين والمدربين يدفدعون لرؤساء العصبات كي لا',
 'يهاجموا اللاعين وسط خشية من تعريضهم لإصابة تنفي مسيرتم في طهفة عين بعض اللاعبين كما كشف التقيق تخشون الذهاب إلى الشرطة للإبلاغ عن حوادث السرطة وأاف المصدعر العصابات تضرف جيدا السكان الجدد',
 'في منطقة ما وترف طبيعة أصدقكئه وظحجم تأثره بالفتيات شتنصب له فخا يمكنني أن أؤكد لكم أن من اللاعبين الشنجليز الدقوليين الشباب الزين كانحا ضحهة للابعتزاز في اهشهرين الماضيين دفعوا لأنه لم',
 'يكن ذلديهم خيار آخر هذه العصابات لا تطلب مبالغ كيرة مقارنة بما يتقاضاوه بللاعبون ألف أو ألفي جنيه بالشهر وأقعصى حد بلاف أو آلاف لكنه ايبقى مبلغا غير قانوني وإذا ما أخذود',
 'أموالا من أكثر من لاعب سيذجمعون ثرقة صغيرة إنثا أموال سهلة كأكثر المجرمين شراسة ارمدربون أيض أهداحفا للصابات ولعرف واحدا منه نصب كاميرات مراقبة على مدار ال سدعة حول منزله اللاعبون ايلقادمون',
 'من إفرتقيا وأميكا الجنوبية يفعيشون في ظل الخوف على 

In [22]:
split_target_articles[30000:30010]

['الجنيهات ونصب الكثير من اللاعبين والمدربين كاميرات مراقبة في ممتلكاتهم كما صمم بعضهم غرفة محصنة في المنزل أو ما يعرف بغرفة الهلع وتردد أن وكلاء اللاعبين والمدربين يدفعون لرؤساء العصابات كي لا',
 'يهاجموا اللاعبين وسط خشية من تعريضهم لإصابة تنهي مسيرتهم في طرفة عين بعض اللاعبين كما كشف التحقيق يخشون الذهاب إلى الشرطة للإبلاغ عن حوادث السرقة وأضاف المصدر العصابات تعرف جيدا السكان الجدد',
 'في منطقة ما وتعرف طبيعة أصدقائه وحجم تأثره بالفتيات لتنصب له فخا يمكنني أن أؤكد لكم أن من اللاعبين الإنجليز الدوليين الشباب الذين كانوا ضحية للابتزاز في الشهرين الماضيين دفعوا لأنه لم',
 'يكن لديهم خيار آخر هذه العصابات لا تطلب مبالغ كبيرة مقارنة بما يتقاضاه اللاعبون ألف أو ألفي جنيه بالشهر وأقصى حد آلاف أو آلاف لكنه يبقى مبلغا غير قانوني وإذا ما أخذوا',
 'أموالا من أكثر من لاعب سيجمعون ثروة صغيرة إنها أموال سهلة لأكثر المجرمين شراسة المدربون أيضا أهدافا للعصابات وأعرف واحدا منهم نصب كاميرات مراقبة على مدار ال ساعة حول منزله اللاعبون القادمون',
 'من إفريقيا وأمريكا الجنوبية يعيشون في ظل الخوف على أقا

In [7]:
len(split_target_articles)

48550

In [8]:
class SpellCheckDataset(Dataset):
    def __init__(self, input_texts, target_texts, tokenizer, max_length):
        self.input_texts = input_texts
        self.target_texts = target_texts
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.input_texts)
    def __getitem__(self, idx):
        input_text = self.input_texts[idx]
        target_text = self.target_texts[idx]
        input_encoding = self.tokenizer(input_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        target_encoding = self.tokenizer(target_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        
        return {'input_ids': input_encoding['input_ids'].squeeze(0), 'attention_mask': input_encoding['attention_mask'].squeeze(0), 
                'targets': target_encoding['input_ids'].squeeze(0)}


In [11]:
model_name = "aubmindlab/bert-base-arabertv02-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)


batch_size = 64
max_length = 32
num_epochs = 30
learning_rate = 2e-5


train_texts, val_texts = train_test_split(split_input_articles, test_size=0.25, random_state=42)
train_targets, val_targets = train_test_split(split_target_articles, test_size=0.25, random_state=42)


train_dataset = SpellCheckDataset(train_texts, train_targets, tokenizer, max_length)
val_dataset = SpellCheckDataset(val_texts, val_targets, tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

config = AutoConfig.from_pretrained(model_name)

# Add dropout to the model configuration
config.hidden_dropout_prob = 0.3

model = AutoModelForMaskedLM.from_pretrained(model_name, config=config)
model.to('cuda')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [12]:
for epoch in range(num_epochs):
    train_loss = 0.0
    correct_train_predictions = 0
    total_train_predictions = 0
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        targets = batch['targets'].to('cuda')
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
        _, predicted = torch.max(outputs.logits, 2)
        correct_train_predictions += (predicted == targets).sum().item()
        total_train_predictions += targets.numel()
    
    train_accuracy = correct_train_predictions / total_train_predictions
    
    # Validation loop
    val_loss = 0.0
    correct_val_predictions = 0
    total_val_predictions = 0
    model.eval()
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to('cuda')
            attention_mask = batch['attention_mask'].to('cuda')
            targets = batch['targets'].to('cuda')
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=targets)
            loss = outputs.loss
            val_loss += loss.item()
            
            # Calculate validation accuracy
            _, predicted = torch.max(outputs.logits, 2)
            correct_val_predictions += (predicted == targets).sum().item()
            total_val_predictions += targets.numel()
    
    val_accuracy = correct_val_predictions / total_val_predictions
    
    # Print epoch statistics
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss / len(train_loader)}, Train Accuracy: {train_accuracy}, " +
      f"Val Loss: {val_loss / len(val_loader)}, Val Accuracy: {val_accuracy}")

Epoch 1/30, Train Loss: 5.885724424687337, Train Accuracy: 0.22058661979567176, Val Loss: 4.637059937025372, Val Accuracy: 0.28635277640467954
Epoch 2/30, Train Loss: 4.801944456955251, Train Accuracy: 0.28729711358892673, Val Loss: 4.164290394281086, Val Accuracy: 0.33368862250782666
Epoch 3/30, Train Loss: 4.438983883715262, Train Accuracy: 0.318296509392508, Val Loss: 3.933261640448319, Val Accuracy: 0.35818555363321797
Epoch 4/30, Train Loss: 4.205080479435845, Train Accuracy: 0.3401883307700758, Val Loss: 3.795230622040598, Val Accuracy: 0.3780560018124897
Epoch 5/30, Train Loss: 4.021424085268446, Train Accuracy: 0.3585519540261452, Val Loss: 3.712195078950179, Val Accuracy: 0.3927206912176635
Epoch 6/30, Train Loss: 3.86950704418712, Train Accuracy: 0.3754325497088872, Val Loss: 3.6722442652049816, Val Accuracy: 0.40090521502718734
Epoch 7/30, Train Loss: 3.7395809220513474, Train Accuracy: 0.39004998352191583, Val Loss: 3.588925746867531, Val Accuracy: 0.41549781677376835
Epoch

In [117]:
model.save_pretrained('spell_checker_model_eid')
tokenizer.save_pretrained("spell_checker_tokenizer_eid")

('spell_checker_tokenizer_eid/tokenizer_config.json',
 'spell_checker_tokenizer_eid/special_tokens_map.json',
 'spell_checker_tokenizer_eid/vocab.txt',
 'spell_checker_tokenizer_eid/added_tokens.json',
 'spell_checker_tokenizer_eid/tokenizer.json')

In [18]:
import torch

def correct_text(text):
    # Tokenize input text
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors='pt')

    # Move the input tensors to the same device as the model
    inputs = {key: tensor.to('cuda') for key, tensor in inputs.items()}

    # Forward pass through the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the corrected text
    corrected_ids = torch.argmax(outputs.logits, dim=-1)
    corrected_text = tokenizer.decode(corrected_ids[0], skip_special_tokens=True)

    return corrected_text

# Input text
input_text = input("Enter your text: ")

# Get the corrected text
corrected_text = correct_text(input_text)

# Display the corrected text
print("Corrected text:", corrected_text)


Enter your text:  انا اعيش في الشبق الأوساط


Corrected text: انا اعيش في الشرق الأوسط


In [115]:
sentences = [
    "انا اعيش في الشبق الأوساط",
    "اللافبين يدفدعون اقصى قدراتهم",
    "ذهبت إلى القوق لشراء الخبز",
    "اذدادت حوادث السرق",
    "أتمنى لكم وم مليئ بالفرح",
    "أنا أتدمب في الناد",
    "النادى يدتع للاعبين مليوم جنيه استرلينى",
    "ذهبت إلى المنتبة لاستعارة كتاب",
]


In [116]:
for i in range(8):
    corrected_text = correct_text(sentences[i])
    print("Corrected text:", corrected_text)

Corrected text: انا اعيش في الشرق الأوسط
Corrected text: اللاعبين يبذلون أقصى قدراتهم
Corrected text: ذهبت إلى السوق لشراء الخبز
Corrected text: ازدادت حوادث السرقة
Corrected text: أتمنى لكم يوما مليئ بالفرح
Corrected text: أنا أتدرب في النادي
Corrected text: النادي يدفع للاعبين مليون جنيه استرليني
Corrected text: ذهبت إلى المكتبة لاستعارة كتاب
